In [ ]:
import pandas as pd
import numpy as np
import sys, gc, os

In [ ]:
os.listdir()

In [ ]:
df_tcgabiolinks = pd.read_csv("../TCGA_biolinks.csv", index_col=0).dropna(how="all", axis=1)
df_tcgabiolinks = df_tcgabiolinks[df_tcgabiolinks["cancer.type"].isin(["LUAD","LUSC"])].set_index("pan.samplesID")
df_tcgabiolinks

In [ ]:
df_files.columns

In [ ]:
df_files=pd.read_csv("files_tcga.dat", index_col=0)
df_files=df_files.drop_duplicates("cases.0.submitter_id")
df_files.sort_values('cases.0.submitter_id', inplace=True)

bins = np.linspace(0,5,num=4)
cutted = pd.cut(df_files["cases.0.samples.0.longest_dimension"].replace("unknown", np.nan).dropna().astype(float),bins=bins, labels=bins[1:])
df_files["longest_dimension"]=df_files["cases.0.samples.0.longest_dimension"]
df_files["longest_dimension"][cutted.index]=cutted

df_files["Age.at.diagnosis"] = df_files["Age.at.diagnosis"].replace("[Not Available]",np.nan)
df_files["Age.at.diagnosis"] = df_files["Age.at.diagnosis"].astype(float)
bins = np.linspace(df_files["Age.at.diagnosis"].min(),df_files["Age.at.diagnosis"].max(), 4)
df_files["Age.at.diagnosis"] = pd.cut(df_files["Age.at.diagnosis"],bins,labels=bins[1:])


df_files=df_files[df_files['cases.0.submitter_id'].isin(df_tcgabiolinks.index)]

In [ ]:
pd.read_csv("../TCGA_biolinks.csv", index_col=0)["cancer.type"].unique()

In [ ]:
df_files = df_files.reset_index().set_index('cases.0.submitter_id').transpose().append(df_meta.reset_index().set_index('patient').reindex(index=df_files['cases.0.submitter_id']).transpose()).transpose().reset_index().set_index("file_name")

In [ ]:
df_files = df_files.reset_index().set_index('cases.0.submitter_id')
df_files = df_files.transpose().append(df_tcgabiolinks.reindex(index=df_files.index).transpose()).transpose()

In [ ]:
df_files.dropna(how="any", axis=1, thresh=len(df_files.index)/10, inplace=True)

In [ ]:
df_files=df_files.fillna("unknown").set_index("file_name")
df_files.to_csv("files.dat", index=True, header=True)

In [ ]:
df = pd.read_csv("mainTable_all.csv",index_col=0)

In [ ]:
df.columns[~df.columns.isin(df_files.index)]

In [ ]:
df = df.reindex(columns=df.columns[df.columns.isin(df_files.index)])
df.to_csv("mainTable.csv", index=True, header=True)
df_files.fillna("unknown").to_csv("files.dat", index=True)

In [ ]:
import scanpy as sc

In [ ]:
adata = sc.AnnData(X=df.transpose(), obs=df_files.reindex(index=df.columns))
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=3000)
sc.pl.highly_variable_genes(adata)
hvg = adata.var[adata.var["highly_variable"]==True].index

In [ ]:
df.reindex(index=hvg).to_csv("mainTable.csv", index=True, header=True)

In [ ]:
from sbmtm import sbmtm

In [ ]:
hsbm = sbmtm()

In [ ]:
hsbm.make_graph_from_BoW_df(df.reindex(index=hvg))

In [ ]:
hsbm.save_graph("graph_hv.xml.gz")

In [ ]:
del hsbm
gc.collect()